## Weak 36

In [ ]:
import tiktoken
from tokenizers import Tokenizer
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
)
from datasets import load_dataset
import polars as pl
import os

In [ ]:
# Get huggingface dataset
dataset = load_dataset("coastalcph/tydi_xor_rc")
df_train = pl.from_pandas(dataset["train"].to_pandas())
df_test = pl.from_pandas(dataset["test"].to_pandas())

In [ ]:
df_train.head()

In [ ]:
df_ar = df_train.filter(pl.col("lang") == "ar")
df_ko = df_train.filter(pl.col("lang") == "ko")
df_te = df_train.filter(pl.col("lang") == "te")
df_arkote = df_train.filter(pl.col("lang").is_in(["ar", "ko", "te"]))
assert df_ar.height + df_ko.height + df_te.height == df_arkote.height; # sanity check

In [ ]:
df_te.describe()

In [ ]:
df_arkote.describe()

### Get tokenizers and look at sample (Arabic) sentence

In [ ]:
# Load multilingual BERT tokenizer
mbert_tokenizer = Tokenizer.from_pretrained("bert-base-multilingual-uncased")
# Load GPT-4 tokenizer
gpt4_tokenizer = tiktoken.get_encoding("cl100k_base")
# Load NLLB-200 tokenizer
nllb_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
# BERT tokenization example
" | ".join(mbert_tokenizer.encode(df_ar["question"][0]).tokens)

In [ ]:
# Decode each token from GPT-4 tokenizer
" | ".join([gpt4_tokenizer.decode([token]) for token in gpt4_tokenizer.encode(df_ar["question"][0])])


In [ ]:
# NLLB-200 tokenization example
" | ".join(nllb_tokenizer.tokenize(df_ar["question"][0]))

## Get the top 5 most frequent words

In [ ]:
# Language dict
lang_dict = {
    "ar": "arb_Arab",
    "ko": "kor_Hang",
    "te": "tel_Telu",
}

In [ ]:
# Set up translation pipeline for NLLB-200
translator_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [ ]:
# Example translation
random_txt = df_ko["question"][0]
translator = translator = pipeline(
        "translation",
        model=translator_model,
        tokenizer=nllb_tokenizer,
        src_lang="kor_Hang",
        tgt_lang="eng_Latn",
    )
translator(random_txt)[0]["translation_text"] 

In [ ]:
def _make_translator(src_lang:str):
    translator = pipeline(
        "translation",
        model=translator_model,
        tokenizer=nllb_tokenizer,
        src_lang=src_lang,
        tgt_lang="eng_Latn",
    )
    return translator

def tokenize_question(df: pl.DataFrame) -> pl.DataFrame:
    
    # Check if already tokenized and translated
    src_lang = lang_dict[df["lang"][0]]
    cache_path = os.path.join("data", f"tydi_xor_rc_{src_lang}")
    print(f"Tokenizing and translating {src_lang}...")
    if f"tydi_xor_rc_{src_lang}.parquet" in os.listdir():
        # Load from parquet
        print(f"Loading from cached file tydi_xor_rc_{src_lang}.parquet")
        return pl.read_parquet(f"tydi_xor_rc_{src_lang}.parquet")

    # Tokenize questions using multilingual BERT tokenizer
    df = df.with_columns(
        pl.col("question")
        .map_elements(lambda x: nllb_tokenizer.tokenize(x), return_dtype=pl.List(pl.Utf8))
        .alias("tokens")
    )

    # Translate questions using NLLB-200
    translator = _make_translator(src_lang)
    df = df.with_columns(
        pl.col("question")
        .map_elements(lambda x: translator(x)[0]["translation_text"])
        .alias("translation")
    )

    # Save to parquet and excel
    df.write_parquet(cache_path + ".parquet")
    df.write_excel(cache_path + ".xlsx")

    return df

In [ ]:
# Tokenize and translate each language dataframe, then compute token frequencies
for df in [df_ar, df_ko, df_te]:
    df = tokenize_question(df)
    count_dict = {}
    for tokens in df["tokens"]:
        for token in tokens:
            if token in count_dict:
                count_dict[token] += 1
            else:
                count_dict[token] = 1
    sorted_frequency_list = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    print(sorted_frequency_list[:10])

In [ ]:
# Tokenize and translate each language dataframe, then compute token frequencies of the translations
for df in [df_ar, df_ko, df_te]:
    df = tokenize_question(df)
    count_dict = {}
    for translation in df["translation"]:
        for token in nllb_tokenizer.tokenize(translation):
            if token in count_dict:
                count_dict[token] += 1
            else:
                count_dict[token] = 1
    sorted_frequency_list = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    print(sorted_frequency_list[:10])

In [ ]:
# Top tokens in answerable true and false
for df in [df_ar, df_ko, df_te]:
    df = tokenize_question(df)
    for answerable in [True, False]:
        count_dict = {}
        for translation in df.filter(pl.col("answerable") == answerable)["translation"]:
            for token in nllb_tokenizer.tokenize(translation):
                if token in count_dict:
                    count_dict[token] += 1
                else:
                    count_dict[token] = 1
        sorted_frequency_list = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
        print(f"Top tokens in {lang_dict[df['lang'][0]]} answerable={answerable}: {sorted_frequency_list[:10]}")